# <center>Implementing LDA in Python for ELOHP Interviews</center>

MOdified tutorial by:

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Importing the Required Libraries

In [27]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob
import os

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

#spacy
import spacy
from nltk.corpus import stopwords


#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [28]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        #data = json.load(f) 
        data=f.read()
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [29]:
#data = load_data("ushmm_dn.json")["texts"]
text_list=[f for f in os.listdir("elohp-interview-text") if not f.startswith('.')]
text_list=sorted(text_list)
data=[]
for f in text_list:
    file_data=load_data("elohp-interview-text/"+f)
    data.append(file_data)
    #data = load_data("elohp-interview-text/Coll520_do001_interviewtext.txt")

#print(data[0][0:90])
full_data_hold=data

In [30]:
print(data[0][800:1500])
avg_length = sum(map(len, data))/float(len(data))
print("AVerage length of interview: " + str(avg_length))

d and transcribed contribute.  
Agapito:  I do.   
Long:  Thank you. Let's just begin with the basic question. Can you please 
tell us when and where you were born and something about your 
early background?  
Agapito:  I was born in Torrance, California, which is near Los Angeles, and 
in 1950. I came from Italian, English, Irish, Welsh, Scottish family, 
Italian on one side and the other side a mixture. And we're a really 
close family. Both sides of my family lived in the same city and we 
were always together. We did a lot of— Every Sunday we would 
go to the Italian aunt's house and have— I'd watch the aunts 
making pasta in the kitchen and just have such great memories of 
all my famil
AVerage length of interview: 67570.43661971831


Splitting interviews into smaller chunks of text to optimize topic modeling

In [99]:
# Python3 code to demonstrate working of
# Divide String into Equal K chunks
# Using len() + loop

data=full_data_hold
chunk_dict={}
def splitchunks(full_string):
    # initializing K - how many chunks of text you want to split it into
    K = 10
    # compute chunk length
    chnk_len = len(full_string) // K
    res = []
    for idx in range(0, len(full_string), chnk_len):
        # appending sliced string
        res.append(full_string[idx : idx + chnk_len])
    #merge the remainder (ie. if it's not evenly divisible combine the last bit of text with the last major section )
    if len(res)>K:
        res[-2]=res[-2]+" "+res[-1]
        res.pop(-1)
    return res

list_chunks=[]
for text in full_data_hold:
     
    text_chunks=splitchunks(text)
    list_chunks=list_chunks+text_chunks

avg_length = sum(map(len, list_chunks))/float(len(list_chunks))
print(avg_length)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
6757.130985915493


Install necessary model

In [32]:
#!python3 -m spacy download en_core_web_sm

In [33]:
#choose whether you want the full length of texts or the split chunked data
#data=full_data_hold
data=list_chunks

Next cell takes a few minutes to load

In [34]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data)

In [35]:
len(lemmatized_texts)

710

In [36]:
print (lemmatized_texts[0][0:200])

long interview part recording make available oral history interview take place record studio interviewer library aggie let know agree record project give permission preserve make available record tran


In [37]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

In [38]:
print (data_words[0][0:200])

['long', 'interview', 'part', 'recording', 'make', 'available', 'oral', 'history', 'interview', 'take', 'place', 'record', 'studio', 'interviewer', 'library', 'aggie', 'let', 'know', 'agree', 'record', 'project', 'give', 'permission', 'preserve', 'make', 'available', 'record', 'transcribed', 'contribute', 'long', 'thank', 'let', 'just', 'begin', 'basic', 'question', 'please', 'tell', 'bear', 'early', 'background', 'bear', 'come', 'italian', 'english', 'irish', 'scottish', 'family', 'italian', 'side', 'other', 'side', 'mixture', 'really', 'close', 'family', 'side', 'family', 'live', 'same', 'city', 'always', 'together', 'lot', 'go', 'italian', 'aunt', 'house', 'watch', 'aunt', 'make', 'pasta', 'kitchen', 'just', 'such', 'great', 'memory', 'family', 'grandmother', 'mother', 'side', 'take', 'care', 'brother', 'lot', 'anyway', 'family', 'really', 'close', 'really', 'fun', 'grow', 'brother', 'brother', 'year', 'young', 'almost', 'year', 'young', 'name', 'really', 'close', 'really', 'close',

In [39]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=150)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=150)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

In [40]:
print (data_bigrams_trigrams[0][:100])

['long', 'interview', 'part', 'recording', 'make', 'available_oral_history', 'interview', 'take', 'place', 'record_studio', 'interviewer', 'library', 'aggie', 'let', 'know', 'agree_record_project', 'give_permission_preserve', 'make', 'available_record', 'transcribed', 'contribute', 'long', 'thank', 'let', 'just', 'begin_basic_question_please', 'tell', 'bear', 'early', 'background', 'bear', 'come', 'italian', 'english', 'irish', 'scottish', 'family', 'italian', 'side', 'other', 'side', 'mixture', 'really', 'close', 'family', 'side', 'family', 'live', 'same', 'city', 'always', 'together', 'lot', 'go', 'italian', 'aunt', 'house', 'watch', 'aunt', 'make', 'pasta', 'kitchen', 'just', 'such', 'great', 'memory', 'family', 'grandmother', 'mother', 'side', 'take', 'care', 'brother', 'lot', 'anyway', 'family', 'really', 'close', 'really', 'fun', 'grow', 'brother', 'brother', 'year', 'young', 'almost', 'year', 'young', 'name', 'really', 'close', 'really', 'close', 'raiskin', 'cousin', 'group', 'c

In [41]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0][0:20])
print(id2word[1])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 4), (11, 4), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 2), (19, 2)]
activity


Processing stopwords

In [42]:
stopword_list = stopwords.words("english")
extra_stopwords=open("extra-stopwords.txt", 'r').read().split()
print(extra_stopwords)
stopword_list = stopword_list+extra_stopwords
print(stopword_list)

['long', 'raiskin', 'stockford', 'drescher', 'rubin', 'laskaya', 'pitt', 'bole', 'grime', 'ree', 'gani', 'hunt', 'anyway', 'interview', 'sort', 'believe', 'guess', 'definitely', 'number', 'literally', 'important', 'go', 'know', 'woman', 'year', 'remember', 'get', 'think', 'say', 'really', 'people', 'time', 'come', 'thing', 'lot', 'want', 'kind', 'live', 'back', 'take', 'mean', 'make', 'see', 'start', 'talk', 'little', 'feel', 'way', 'tell', 'call', 'first', 'still', 'much', 'move', 'right', 'part', 'happen', 'also', 'life', 'together', 'different', 'look', 'place', 'never', 'actually', 'group', 'always', 'even', 'try', 'big', 'find', 'maybe', 'need', 'day', 'like', 'meet', 'great', 'let', 'name', 'well', 'many', 'leave', 'stuff', 'become', 'pretty', 'give', 'high', 'hard', 'end', 'probably', 'use', 'put', 'point', 'whole', 'change']
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he'

In [43]:
from nltk import FreqDist
alltexts_no_stopwords=[]
for text in data_bigrams_trigrams:
    new_word_list=[]
    for word in text:
        if word not in stopword_list:
            new_word_list.append(word)
    alltexts_no_stopwords=alltexts_no_stopwords+new_word_list
fdist = FreqDist(alltexts_no_stopwords)
fdist.most_common(25)
most_common_list=[]
for word in fdist.most_common(50):
    most_common_list.append(word[0])
for word in most_common_list:
    print(word)


work
lesbian
school
community
friend
good
gay
love
man
house
old
job
family
kid
person
couple
class
home
young
ask
child
parent
experience
relationship
play
student
grow
sure
fun
interesting
guy
college
help
run
wonderful
stay
course
mother
town
early
bring
keep
ever
hear
bit
book
teach
partner
mom
story


Remove stopwords - both standard and custom list. 

In [44]:
data_no_stopwords=[]
for text in data_bigrams_trigrams:
    new_word_list=[]
    for word in text:
        if word not in stopword_list:
            new_word_list.append(word)
    data_no_stopwords.append(new_word_list)
id2word=corpora.Dictionary(data_no_stopwords)
texts=data_no_stopwords
corpus = [id2word.doc2bow(text) for text in texts]
#for text in corpus:
    #print(id2word[text[0][0]])

In [45]:
#TF-IDF REMOVAL

#id2word = corpora.Dictionary(data_bigrams_trigrams)
#texts = data_bigrams_trigrams

#print (corpus[0][0:20])
tfidf = TfidfModel(corpus, id2word=id2word)
corpus = [id2word.doc2bow(text) for text in texts]

#low_value = 0.03 #threshold number
low_value = 0.05 #threshold number
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    print("\nnew text ****\n")
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    for id, value in tfidf[bow]:
        if value < low_value:
            print(id2word[id]+ " "+ str(value))
#            next
        else:
#            print(id2word[id]+" "+str(value))
            next 
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        #print(item)
        words.append(id2word[item])
        #print(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf score 0 will be missing
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    for nb in new_bow:
        print(id2word[nb[0]])
    corpus[i] = new_bow


new text ****

activity 0.03161774399284589
adult 0.036501485247966176
age 0.0381413906180411
agree_record_project 0.03238868919818853
almost 0.021226209128109488
available_oral_history 0.0329959588538497
background 0.027571142084025325
backyard 0.04498336624239994
bad 0.020527122209284935
bear 0.041744646134173335
bedroom 0.04359868563593874
bit 0.012692021578008963
care 0.018917800010237268
check 0.032191955515794415
childhood 0.03911924252976018
choir 0.047645287544759315
city 0.048640543387119636
clothe 0.040816023429183126
color 0.039773603722524704
contribute 0.04404527009672012
cry 0.04233818049108653
dad 0.025483432304614476
decide 0.029477529140832594
difficult 0.02985717056233935
dinner 0.03238868919818853
drop 0.033848718667046016
early 0.013444405669163068
elementary 0.04498336624239994
father 0.023559749778056742
finish 0.03258821352219379
follow 0.02921033342912563
friend 0.014881741448230536
fun 0.02836980895902602
garden 0.04547696878259687
good 0.010165917209692845
gr

job 0.01518200267491131
miss 0.02406654127381205
next 0.022431576860184686
raise 0.016561406028733232
seem 0.01240679246511887
sex 0.020790843971927778
sure 0.008212555907395213
word 0.015990132975809004
accept 0.021851401439405522
explain 0.023465408297958425
fill 0.02556673131570451
show 0.013537836521183927
summer 0.0356230092958901
truck 0.022898160431901092
bag 0.03644718345461407
camera 0.03699142334282975
line 0.02035312344727154
mind 0.018770895576197544
person 0.02093162928676464
set 0.017012197142368637
sock 0.04649836060246915
chop_wood 0.048050039175669315
question 0.025107487549177004
sneak 0.042908075637606895
story 0.011129753838624442
totally 0.01933549135794939
anniversary 0.034524024130797984
center 0.03809847100265337
education 0.020790843971927778
essay 0.04515423548399912
hand 0.01663514984517318
student 0.012803532917630641
afraid 0.02422260619723448
campus 0.03496825331611563
carry 0.025748105338452423
invite 0.020903314258464718
sense 0.015783996367576148
stick 

process 0.03308142437613985
funny 0.032279689844834265
story 0.03846561535141457
totally 0.033412759347128757
science 0.0458075186531538
student 0.022125187110590396
campus 0.03021350249680753
coast 0.0448132731920711
deal 0.02358317603077067
member 0.03409498639671533
pass 0.02775217986581653
sad 0.04357027595299685
involve 0.03936538412679368
course 0.017812054261634236
guy 0.037444477723351285
idea 0.02006669353057314
health 0.03909908947716402
issue 0.023022012526113318
share 0.03275628294626522
nice 0.04932298792230202
office 0.033750528407347914
theater 0.0441806740221849
director 0.040803415572705115
challenge 0.03980917011162241
consider 0.040052454637325444
discover 0.04960394341028823
faculty 0.04918477710263359
heterosexual 0.0438727976657234
major 0.03886869458527471
path 0.04877547445276277
positive 0.04722827206722685
problem 0.030923583131265933
professional 0.0479846764575205
sexuality 0.040803415572705115
army
bedroom
fun
horse
patch
shoe
wake
wear
white
class
ear
expl

begin
community
date
pay
short
teach
transition
completely
experience
sexual
support
chair
learning
simple
build
burn
develop
food
eventually
career
degree
lead
practice
stick
table
therapist
therapy
bottom
willing
anywhere
folk
supervisor
crisis
incest
hierarchy
contract
organize
self
abuse
former
pocket
challenge
discover
satisfying
act
coursework
decision
shit
flyer
highlight
licensing
sweetie
actualize
advisor
clientele
committee
digression
facilitate
fading
finish_degree
grandfathere
key
midlife
plus
recover
rounded
spanish
suggest
survival
thesis
timer

new text ****

care 0.030286135211674343
decide 0.023595778388137468
family 0.017049712159905014
good 0.005424984987627765
home 0.018444529055008325
old 0.012255828115763557
school 0.044031266469351044
away 0.026180418969228042
community 0.020139626019639388
ever 0.04001693269017971
lesbian 0.004322431526611771
man 0.014738033633841418
marry 0.032862552643570825
month 0.02711237653466836
okay 0.03471058102055221
work 0.01125977449

official
environmental
accountant
finance
absence
account
blatantly
broaden
budget
cake
centerfold
coverage
custody
dessert
domestic_partner
effect
encounter
espresso
fundamentalist
homophobia
inaudible
ironic
joint
lab
laboratory
lunchroom
menu
miser
oblivious
operation
overlook
partn
profession
pumpkin
quality
restricted
technician
treatment
tune
twilight
waitressing
wastewater
wednesday

new text ****

activity 0.03892808471078952
bit 0.015626544741829
care 0.023291785824659822
child 0.040898286536169226
family 0.013112212609006954
friend 0.04275261641166926
good 0.02503275922477832
grow 0.0338755172407672
hear 0.03348850801347194
home 0.014184907297791158
house 0.025708216475846236
middle 0.031525402914243446
parent 0.038593425405843595
roll 0.04816400522962659
school 0.03386258500877744
walk 0.020449143268084613
weekend 0.04337971589239579
world 0.04137825216761586
away 0.040268506177444964
community 0.023232802034313462
ever 0.030775330660883143
fact 0.04910726765517223
last 0.01

...

bit 0.035460215553911205
early 0.012520766690385036
good 0.006311674942898775
hear 0.012665518112031105
home 0.010729603135820913
kitchen 0.03835221289794222
learn 0.030815957952186387
love 0.008223165447953129
new 0.016662513018465868
parent 0.014596222927769551
play 0.033063026446201606
value 0.03449771394414326
wonderful 0.013261069319116445
world 0.015649458078642993
away 0.015229746697872937
community 0.011715679692884718
ever 0.011639381119408385
exciting 0.028764919594796375
hang 0.022341207531285247
huge 0.02339822180890064
kill 0.035849414062978195
lesbian 0.02514456982012865
man 0.00857345023133352
mostly 0.018803111874413164
okay 0.04038387294720924
sit 0.031790944528255365
turn 0.03293300265209771
view 0.03475712579469443
work 0.008733423437982769
bring 0.012425146774089747
enough 0.017063560779105504
experience 0.011372933892549942
feeling 0.02275383598656819
gay 0.03529737844241316
incredible 0.03399381065171073
job 0.009878926341836078
keep 0.01228301297066985
next 0.014

person 0.005641877627497707
piece 0.014538314402522156
wet 0.0365123996045099
empty 0.03139583384255813
head 0.014816615319321812
area 0.028275859946785747
couple 0.0056648381755143445
stand 0.013881546293328413
university 0.014269215220751254
chance 0.049470379404453625
control 0.024735189702226813
doctor 0.01818799003067828
less 0.03380547384238947
ready 0.01862807065802688
skin 0.03139583384255813
yet 0.014269215220751254
advice 0.02159608651709855
extremely 0.024735189702226813
plan 0.015634971546825752
course 0.016669737370559257
guy 0.008760775153409422
hit 0.01958676672201678
paper 0.01829579967135475
send 0.0286713211202879
sister 0.042609923718612636
idea 0.028169674266936368
review 0.03555370354052
embarrassed 0.030377067329724523
awful 0.020673646677307533
bed 0.021276849174323462
horrendous 0.040337998315799636
horrible 0.017373359831563732
floor 0.018081589706752707
upset 0.02341236543330652
quick 0.028657115086559153
curtain 0.0365123996045099
desk 0.027916649490525195
re

twenty_thirty
sadly
compensation
determined
comp
island
ass
accomplish
formulate
remarry
calling
dawning
seminal
waterworks

new text ****

bear 0.029149761364379988
bit 0.01772535807544697
church 0.04854009870803359
dinner 0.045233228615603596
early 0.018776118771369803
friend 0.04156693153014442
fun 0.019810280783088843
good 0.014197464257338753
hear 0.018993187737860358
home 0.032180186378289594
house 0.014580553476281295
kid 0.020039151817574166
least 0.03645505883755411
mom 0.028667662529700332
moment 0.03972067744006793
morning 0.040143156982897346
old 0.032074131451878506
parent 0.0437769382789712
run 0.020116038575553306
school 0.012803565014048644
strong 0.034923540094010566
tree 0.04758163829899868
write 0.04823626390787848
away 0.022838500223531094
college 0.025793116949221793
community 0.026353217679136082
date 0.03396507968497566
ever 0.01745439458520335
exciting 0.04313582068225491
graduate 0.029028122947342798
last 0.02155408589015679
lesbian 0.03770675079827861
man 0.03

college 0.023798511313671798
divorce 0.04669615054150113
fight 0.04003098777367809
full 0.03299428087827992
last 0.019887288454613333
lesbian 0.02087451169026032
phone 0.03534716968727621
room 0.025697672866975108
sit 0.021993497600299786
sudden 0.04636356486198503
teach 0.02447527193544285
though 0.036079609177219606
wife 0.03890482370610609
experience 0.015735964941328028
figure 0.023798511313671798
raise 0.029821449482426033
reason 0.03105308315597713
relationship 0.017524426929557257
second 0.027351592538789275
sure 0.014788015020528957
interesting 0.015675243239708835
later 0.017524426929557257
laugh 0.049212618163682796
light 0.04738020949676375
period 0.034816631978503425
road 0.04173530113020418
show 0.02437703098501406
sorry 0.040264867289388544
southern 0.046036972044478035
week 0.024974626133760284
person 0.012563577518260314
main 0.04809110104165128
story 0.04008178910347785
couple 0.012614707061280324
ask 0.027774376204757496
deal 0.02457404826154126
therapist 0.0492126181

originally
uniform
organize
theme
birthday
lace
ballroom
biology
chuckle
dykey
facility
friendly
purpose
socialize
shake
butch
plaid
flyer
kinda
respect
ninety
chemistry
tough
campground
product
avenue
grab
variety
planning
extreme
wholesale
mid
alcoholic
themed
ugly
distribution
nail
latin
recovery
flannel_shirt
technology
dip
butchy
lipstick
hysterical
campsite
manufacture
associat
autonomy
campout
command
hazard
migrating
regulatory
winner

new text ****

care 0.03949947524809272
friend 0.010357458366631803
good 0.014150637494181018
mother 0.03810434958291092
college 0.03856206726544026
community 0.026266298222229034
lesbian 0.011274715370825186
okay 0.04526988096321127
pay 0.04625223419410021
work 0.04895036193725458
around 0.0280712857027271
finally 0.04268220368757638
gay 0.019783988059915485
stay 0.027223006562971386
visit 0.049191647691910374
later 0.028395815231419867
describe 0.04605311733979602
hold 0.04875328696434309
lose 0.045463740568147774
question 0.03662817841826705
a

## Generating the Model

In [53]:
num_topics = 20

In [50]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=70,
                                           alpha="auto")

lda_model.save("models/num_topics_"+str(num_topics)+".model")

https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know

## Vizualizing the Data

In [54]:
lda_model = gensim.models.ldamodel.LdaModel.load("models/num_topics_"+str(num_topics)+".model")

In [55]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/cblevins/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.462451 -0.143124       1        1  21.049057
12     0.432991  0.176545       2        1  16.320171
3     -0.460768  0.047272       3        1   7.180661
7     -0.388993 -0.187027       4        1   7.170689
11     0.176859 -0.406890       5        1   6.656396
4     -0.095832 -0.387107       6        1   5.696534
14    -0.263870 -0.337123       7        1   5.234851
17     0.221625 -0.218209       8        1   4.885389
15     0.090018  0.405114       9        1   3.762324
0      0.032738 -0.273307      10        1   3.476086
13     0.273417 -0.050806      11        1   3.140407
6     -0.132082  0.392294      12        1   3.117040
5      0.251403  0.234664      13        1   3.096667
19    -0.273874  0.238060      14        1   2.265858
16     0.089018  0.224610      15        1   1.981407
18    -0.108149  0.191156      16        1   1.713085
9     -0.234501  0.048394      17        1   1.298201
1     -0.096514 -0.047593      18        1   1.028599
10     0.048526  0.060448      19        1   0.699040
8     -0.024464  0.032627      20        1   0.227537, topic_info=         Term         Freq        Total Category  logprob  loglift
152    school  1198.000000  1198.000000  Default  30.0000  30.0000
409       gay   488.000000   488.000000  Default  29.0000  29.0000
100       kid   419.000000   419.000000  Default  28.0000  28.0000
32      child   383.000000   383.000000  Default  27.0000  27.0000
216     class   461.000000   461.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
547   meaning     0.017313     1.062070  Topic20  -9.4974   1.9691
560    period     0.017314     1.062184  Topic20  -9.4973   1.9690
124       old     0.017314     1.062279  Topic20  -9.4973   1.9690
1741   chorus     0.017313     1.062119  Topic20  -9.4974   1.9691
214    circle     0.017313     1.062258  Topic20  -9.4974   1.9689

[897 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
973      15  0.976099   abortion
1729      8  0.946591   accident
1730     15  0.975178   activism
1200      9  0.981861   activist
0         1  0.990550   activity
...     ...       ...        ...
4021     13  0.977066     writer
863      13  0.952747    writing
192      17  0.994267      young
1890     17  0.892931  youngster
3679     16  0.957266       yurt

[1073 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 13, 4, 8, 12, 5, 15, 18, 16, 1, 14, 7, 6, 20, 17, 19, 10, 2, 11, 9])

For making binder notebook: https://www.youtube.com/watch?v=owSGVOov9pQ

In [91]:
topic_dict = {i: [] for i in range(num_topics)}  # Assuming you have 20 topics. 

#Loop over all the documents to group the probability of each topic
for docID in range(len(corpus)):
    topic_vector = lda_model[corpus[docID]]
    for topicID, prob in topic_vector:
        topic_dict[topicID].append((docID, prob))

for topicID, probs in topic_dict.items():
    doc_probs = sorted(probs, key = lambda x: x[1], reverse = True)
    docs_top_5 = [dp[0] for dp in doc_probs[:5]] 

In [93]:
#Then, you can sort the dictionary to find the top documents:
for topicID, probs in topic_dict.items():
    print("Topic " + str(topicID+1)+" top documents:")
    doc_probs = sorted(probs, key = lambda x: x[1], reverse = True)
    #can specify top scoring document numbers (ex. 5)
    for p in doc_probs[:5]:
        print(str(p))

Topic 1 top documents:
(92, 0.7267653)
(68, 0.50812095)
(32, 0.372207)
(258, 0.3277254)
(41, 0.29305917)
Topic 2 top documents:
(72, 0.23073047)
(79, 0.1922666)
(215, 0.19149719)
(182, 0.17418672)
(355, 0.16248797)
Topic 3 top documents:
(59, 0.81092376)
(97, 0.8033117)
(88, 0.803253)
(98, 0.6893622)
(87, 0.6840705)
Topic 4 top documents:
(46, 0.736441)
(77, 0.51440734)
(45, 0.48379076)
(665, 0.46761337)
(34, 0.4634188)
Topic 5 top documents:
(94, 0.71506315)
(95, 0.67164725)
(18, 0.64156026)
(566, 0.5616994)
(91, 0.52420306)
Topic 6 top documents:
(82, 0.5434546)
(84, 0.524566)
(554, 0.38239452)
(551, 0.3708746)
(153, 0.29526973)
Topic 7 top documents:
(246, 0.3357264)
(310, 0.32365546)
(487, 0.29709834)
(247, 0.28422758)
(680, 0.2753085)
Topic 8 top documents:
(93, 0.6024506)
(15, 0.53430915)
(67, 0.4847168)
(702, 0.39576358)
(10, 0.38731843)
Topic 9 top documents:
(184, 0.14429308)
(217, 0.13262096)
(87, 0.119003266)
(395, 0.103931084)
(216, 0.06638894)
Topic 10 top documents:
(259,

In [97]:
for d in data:
    print(d[0:200])

 
 
Long:  This interview is part of the Eugene Lesbian Oral History Project. 
The recordings will be made available through the University of 
Oregon Libraries’ Special Collections and University Arc
 eighth grade, all in the same school 
and that was Meadow Park Elementary. And then you'd go to high  
school in eighth grade and that was South Torrance High. And my 
mother made all my clothes, alm
hen I remember seeing the 
maid who was cleaning rooms. We both looked at each other. And 
we went— because she knew Dolly Parton was there too.  
 And another delivery was to some agent and he had a 
y, 1970, that's when I came out as a lesbian. And I 
remember early times of stopping racist comments and jokes. I was 
living in Los Angeles, and I had some black friends, and there was  
an early un
 candy. And that was as far as Ruth could give us a ride.  
 So we had to hitchhike to the airport, but we decided to hitchhike 
to WomanShare  if I remember this right. And we finally visited 
Wo

In [89]:
import pandas as pd

##dominant topic for each document
def format_topics_sentences(ldamodel=lda_model, 
                            corpus=corpus, 
                            texts=data_bigrams_trigrams, 
                            n=1):
    """
    A function for extracting a number of dominant topics for a given document
    using an existing LDA model
    """
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            # we use range here to iterate over the n parameter
            if j in range(n):  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(
                    # and also use the i value here to get the document label
                    pd.Series([int(i), int(topic_num), round(prop_topic, 4), topic_keywords]),
                    ignore_index=True,
                )
            else:
                break
    sent_topics_df.columns = ["Document", "Dominant_Topic", "Perc_Contribution", "Topic_Keywords"]

    # Add original text to the end of the output
    text_col = [texts[int(i)] for i in sent_topics_df.Document.tolist()]
    contents = pd.Series(text_col, name='original_texts')
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df

In [90]:
format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_bigrams_trigrams, n=2)

,Document,Dominant_Topic,Perc_Contribution,Topic_Keywords,original_texts
0,0.0,19.0,0.0336,"child, baby, daughter, cool, adopt, pregnant, ...","[long, interview, part, recording, make, avail..."
1,0.0,17.0,0.0353,"class, teach, middle, sign, department, surger...","[long, interview, part, recording, make, avail..."
2,1.0,16.0,0.0118,"law, doctor, office, lawyer, person, nurse, ho...","[eighth_grade, same, school, then, go, high, s..."
3,1.0,14.0,0.0212,"job, fun, land, marry, city, wedding, beautifu...","[eighth_grade, same, school, then, go, high, s..."
4,2.0,19.0,0.0167,"child, baby, daughter, cool, adopt, pregnant, ...","[remember, see, maid, clean, room, look, other..."
...,...,...,...,...,...
1415,707.0,14.0,0.1686,"job, fun, land, marry, city, wedding, beautifu...","[party, raiskin, know, summarize, say, generat..."
1416,708.0,19.0,0.0142,"child, baby, daughter, cool, adopt, pregnant, ...","[ound, too, busy, organize, party, mostly, jus..."
1417,708.0,16.0,0.0141,"law, doctor, office, lawyer, person, nurse, ho...","[ound, too, busy, organize, party, mostly, jus..."
1418,709.0,15.0,0.0219,"straight, movement, seem, hire, night, quite, ...","[think, struggle, lesbian, community, pandemic..."


In [139]:
print(corpus[0][0:100])
print(id2word[2])
print(data[0][1000:2000])
print(lemmatized_texts[0][1000:2000])

[(3, 2), (9, 6), (16, 2), (20, 4), (26, 1), (27, 4), (28, 3), (34, 2), (38, 1), (42, 3), (61, 1), (65, 3), (72, 2), (76, 1), (86, 2), (95, 3), (102, 2), (112, 3), (118, 8), (119, 3), (132, 1), (134, 2), (140, 1), (141, 3), (144, 2), (153, 1), (154, 12), (156, 1), (161, 1), (164, 3), (167, 2), (169, 3), (170, 2), (194, 10), (203, 2), (215, 24), (218, 2), (220, 3), (225, 4), (234, 2), (237, 1), (245, 2), (252, 2), (255, 1), (258, 6), (266, 1), (267, 1), (272, 2), (275, 2), (277, 1), (280, 8), (285, 1), (286, 4), (288, 3), (307, 5), (320, 4), (322, 6), (323, 11), (325, 1), (327, 7), (329, 1), (333, 2), (334, 1), (336, 7), (345, 4), (348, 1), (349, 4), (353, 2), (354, 5), (358, 2), (361, 5), (362, 2), (363, 1), (368, 4), (373, 5), (378, 3), (380, 2), (394, 14), (395, 4), (397, 2), (399, 4), (401, 2), (410, 4), (413, 2), (414, 4), (418, 2), (419, 2), (423, 6), (424, 2), (433, 12), (434, 2), (441, 1), (444, 4), (445, 2), (448, 4), (452, 6), (458, 8), (462, 2), (467, 2), (475, 22)]
able
ect. 

Saving Models

https://www.youtube.com/watch?v=xADAr8pPQMI&list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX&index=12

In [136]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=30,
                                           alpha="auto")


In [132]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.23957382), (1, 0.114857174), (2, 0.068196125), (3, 0.012150145), (4, 0.22865918), (5, 0.036370933), (7, 0.024131227), (8, 0.035303768), (9, 0.046170343), (10, 0.037606623), (11, 0.01454134), (12, 0.049493156), (15, 0.017285148), (16, 0.019996101), (18, 0.055496957)]
[(0, 0.23957382), (4, 0.22865918), (1, 0.114857174), (2, 0.068196125), (18, 0.055496957), (12, 0.049493156), (9, 0.046170343), (10, 0.037606623), (5, 0.036370933), (8, 0.035303768), (7, 0.024131227), (16, 0.019996101), (15, 0.017285148), (11, 0.01454134), (3, 0.012150145)]


In [20]:
lda_model.save("models/test_model.model")

NameError: name 'lda_model' is not defined

In [134]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [135]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.2395575), (1, 0.11485863), (2, 0.068188906), (3, 0.0121496385), (4, 0.22869575), (5, 0.036369193), (7, 0.024140326), (8, 0.035302002), (9, 0.046137094), (10, 0.037605748), (11, 0.014545072), (12, 0.049502864), (15, 0.017285211), (16, 0.020000387), (18, 0.055493727)]
[(0, 0.2395575), (4, 0.22869575), (1, 0.11485863), (2, 0.068188906), (18, 0.055493727), (12, 0.049502864), (9, 0.046137094), (10, 0.037605748), (5, 0.036369193), (8, 0.035302002), (7, 0.024140326), (16, 0.020000387), (15, 0.017285211), (11, 0.014545072), (3, 0.0121496385)]
